In [1]:
# MIT License
# 
# Copyright (c) 2019 James Bushnell
# 
# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:
#   
#   The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.
# 
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
# SOFTWARE.


In [2]:
using DelimitedFiles
subclusterCosts1 = readdlm("/home/james/subclusterCosts.txt")
clusterDists1 = readdlm("/home/james/clusterDists.txt")



2×200 Array{Float64,2}:
 3.84832   2.95773   2.3614   3.15434   …  1.92491  1.67902  1.31057  1.19374
 0.859528  0.541168  1.45509  0.183592     4.01082  3.0617   2.11898  3.62931

In [3]:
using JuMP
using Cbc
aMIPModel = Model(solver=CbcSolver())

#the nearness of points (measured as the order of nearness of the subcluster center that the point belongs to)
#this is used as the exponent (x) in pen^x when calculating penalty amounts per point per cluster (see below for
#calculation of "pen")
#an example of what subclusterCosts should look like:
#subclusterCosts      = [
#[ 1 2 1 1 1 1 1 1 1 2 1 2 1 1 2 3 2 1 1 1 2 1 1 1 2 1 3 2 1 1 2 2 3 1 1 3 1 2 2 1 2 1 1 4 1 1 1 2 3 1 2 2 2 1 1 1 2 1 2 1 2 1 1 2 1 2 2 1 3 4 1 2 1 2 1 1 2 2 2 1 1 2 1 1 1 1 3 2 2 1 3 1 2 1 1 2 4 1 1 1 4 4 3 4 3 3 1 3 5 3 5 5 4 3 3 5 3 3 3 3 3 5 4 5 3 5 3 3 5 3 5 3 4 3 3 1 3 4 5 3 3 3 5 3 3 4 3 3 3 4 3 3 3 5 4 3 5 4 1 2 3 3 5 5 5 4 3 2 3 4 5 4 4 3 5 3 4 5 4 4 2 5 5 3 5 5 5 4 3 5 4 4 5 3 4 5 4 3 4 4 ] ;
#[ 5 2 5 5 5 5 5 5 5 2 5 2 5 5 2 1 2 5 5 5 2 5 5 5 2 5 1 2 5 5 2 2 1 5 5 1 5 2 2 5 2 5 5 4 5 5 5 2 1 5 2 2 2 5 5 5 2 5 2 5 2 5 5 2 5 2 2 5 1 4 5 2 5 2 5 5 2 2 2 5 5 2 5 5 5 5 1 2 2 5 1 5 2 5 5 2 4 5 5 5 4 4 1 4 1 1 5 1 3 1 3 3 4 1 1 3 1 1 1 1 1 3 4 3 1 3 1 1 3 1 3 1 4 1 1 5 1 4 3 1 1 1 3 1 1 4 1 1 1 4 1 1 1 3 4 1 3 4 5 2 1 1 3 3 3 4 1 2 1 4 3 4 4 1 3 1 4 3 4 4 2 3 3 1 3 3 3 4 1 3 4 4 3 1 4 3 4 1 4 4 ] ;
#[ 5 4 5 5 5 5 5 5 5 4 5 4 5 5 4 3 4 5 5 5 4 5 5 5 4 5 3 4 5 5 4 4 3 5 5 3 5 4 4 5 4 5 5 1 5 5 5 4 3 5 4 4 4 5 5 5 4 5 4 5 4 5 5 4 5 4 4 5 3 1 5 4 5 4 5 5 4 4 4 5 5 4 5 5 5 5 3 4 4 5 3 5 4 5 5 4 1 5 5 5 1 1 3 1 3 3 5 3 2 3 2 2 1 3 3 2 3 3 3 3 3 2 1 2 3 2 3 3 2 3 2 3 1 3 3 5 3 1 2 3 3 3 2 3 3 1 3 3 3 1 3 3 3 2 1 3 2 1 5 4 3 3 2 2 2 1 3 4 3 1 2 1 1 3 2 3 1 2 1 1 4 2 2 3 2 2 2 1 3 2 1 1 2 3 1 2 1 3 1 1 ] 
#]
subclusterCosts = subclusterCosts1

#distances of each point from the cluster centers 
#an example of what clusterDistances should look like:
#clusterDistances   = [
#[ 0.738657880163994 0.686975752668135 1.47657148145252 0.0164045104327921 0.600160836745103 1.65795366153416 0.571931784225668 1.88528464446882 0.773301865347137 1.37864675476564 1.15962301715836 0.999243161076432 1.30344534292065 0.298485803622766 1.08307732989035 1.82240308405529 0.616482100863188 2.07264977536682 0.7908171019114 0.873268398815725 1.24610219093714 0.66764919367917 1.12044800550482 0.820606496887768 2.30907741389422 1.79769125172967 0.954701074471633 0.436222672436383 1.36715657297721 1.20134265637547 1.82953761484286 0.890200371322248 0.900094448135591 0.795203750015286 1.85253487524333 1.6020412804822 1.20241016212234 1.10292958980621 2.29582843153149 1.18640854579481 1.3122788115415 0.307602389590065 1.82001329768039 2.38500465988947 1.67858580555021 1.2220942992247 1.21172081498681 1.17853758481485 2.54940877182909 0.949234315998268 1.15297138867306 1.12789388243003 0.697233156025096 1.4403586153353 0.389535380650631 1.43234272379525 1.87387269258281 0.499021225427525 1.33053583675658 0.131746920780214 1.43608181176286 0.911438430724072 0.999299194374451 3.75996141176572 1.15952006664801 0.686628683641802 1.05380789122521 0.134849249736308 1.20668671889692 2.09260866489747 0.593191571939591 2.43139473520864 0.973502039418583 2.60576570047576 0.865927556636319 1.19785611786274 0.538687495604885 1.46549343512258 0.795281123038436 0.248313926930012 0.714817148353707 1.64368957030406 0.456642864253142 0.757933570102006 0.327940372197419 0.291052676665912 1.77821778973919 0.559984363297347 1.18122066794767 1.07285489310437 1.07034448302236 0.463294004862548 0.472956286383779 0.896522526211907 1.59441339494594 2.44837143713733 2.3076674931804 1.70282650400454 0.412443718783606 1.38938804152061 4.4269378428954 3.60152757878406 2.17246085690413 2.78015715217853 2.43302322147386 3.04502225804686 1.17532096388912 2.88445690670824 5.05912132858806 4.7683275756109 4.18179112670256 3.78504756829621 3.20822993344469 2.2410081935677 2.20297772326209 4.71472329955107 2.68699181638223 1.61742069902883 4.0939500253134 3.32452052843285 2.91653882117216 4.37768012345538 3.18944151763992 3.9945127763303 2.39853273744576 3.6218460610575 2.59170103622685 3.21690169107761 4.63011256804546 2.5255231282746 5.13151884846749 2.16383588955059 2.68185201002159 4.04673362019386 3.04134542360391 0.668510182147119 1.99369363499821 3.17412834310613 3.92056474925422 2.51995454567854 3.18281709487706 3.6003886691235 5.27464864093088 2.53116095598605 2.88182909569835 3.86323523557393 3.04978539962893 1.86539605890654 1.86073347313771 3.23728078767486 3.70269538329442 1.39932704136103 2.48470155399492 3.18077782152017 3.80381217932383 3.24836755510658 4.07568417274109 3.44358291298641 1.80392460335653 4.93920772545645 2.55986757913162 3.32468718667139 3.27814978320289 4.66262753315925 5.26811066736473 4.07138584112977 3.26051098826242 1.4163984746504 3.55389086877176 2.75546073628204 5.51028797826286 2.96097133315709 3.21244797181014 3.22045833370486 3.97939020212292 2.29424400641521 2.49050468197089 3.19248389902565 3.30990552398389 2.01872579939335 2.40027063601163 3.67333954137961 3.41756866271971 2.0980871103502 3.4833686330901 4.25668545615234 3.52850862162502 3.75756420276525 2.55915831524934 3.40887139655347 3.52301458787257 2.97463384029026 4.45412324707404 2.0267184215361 4.30180853389199 4.33579901745683 4.09730651436754 3.24794437201224 2.2220447989198 3.74225434865421 ] ;
#[ 3.20313551509784 2.19929874229366 2.44984984146236 2.61671788974805 3.17773379617799 2.30358699628373 2.94561021961426 4.38238348229625 2.98200224666092 1.83124662907621 2.71317725444122 1.61983571982511 3.77815381056363 2.32858732316966 2.18886883744959 2.01073106584576 2.06344961993986 4.00651136780832 2.98120402331182 3.44693783834037 2.83353966126119 3.27910082844634 3.26675095559034 2.90250522144025 1.58440548779899 3.81686482841927 1.88945402045222 2.18860954331894 3.71604645375378 2.21675137279525 0.84718301529646 2.07462609728453 2.08149689958999 2.54547901189145 4.17713273826296 1.02110643496878 3.60317055554461 1.70419463010386 1.25885531911787 3.80102458460412 2.16170779656305 2.64417391172815 4.29996894042188 3.84074395481623 3.73390249019079 3.35773152283052 3.82556779623447 2.00534356036982 0.156324569062594 3.55814027542283 1.49849117288461 1.66111223551065 2.03967707344012 3.16065559870174 2.52435842729086 2.47217733594902 2.93243051404075 2.51923570710764 1.4033406273939 2.59832519695743 1.20508309849652 3.48190173522887 3.61233338580025 2.25094192731812 3.23629358203218 1.93023615709806 1.56340192067032 2.75123992861478 1.60553968967693 2.07809015517611 2.73819411851122 3.84111655618085 2.15759068200466 1.64391098210212 3.29148878494247 3.21963254258969 2.41506187364466 2.81579545709426 1.84651777378687 2.79512187742504 3.31835427940902 1.01981914418553 2.7954902447022 3.00193822499075 2.62632898242942 2.39660717358333 1.02533440076186 2.09795087938867 1.86049284905983 2.630374177354 1.90881431499782 2.47734365711464 2.14388007929137 3.39828633450762 3.45947837522459 1.54005118670601 1.97091222371373 3.42619962116869 2.97354391687686 3.84548496653583 3.36986501351521 2.53361869593907 1.32881231661925 1.9918757111575 0.858373673617945 0.624974371121729 2.15440666856244 0.479141138504939 2.93589870550135 2.16581214346014 1.67115894572535 1.45462749680673 2.94938818443109 0.9086447725611 0.479952473900477 2.80193309736381 0.664487366317673 1.42277617665414 1.57141080815351 0.81897695318242 0.504624079790779 2.0085153340847 2.62033755377612 1.85285816088754 0.953242992294701 1.25733873770252 0.40455050942812 0.617965487770378 2.29653611316456 0.125341728486985 3.15045814877139 1.63363189272888 1.53305427190165 1.43457665377375 0.553206575145202 2.18909641079451 1.72856740081882 1.51905019074491 1.57597118791431 0.36166907660807 0.573051974147162 1.00849911762268 2.99316714392572 0.0979027315738949 0.90323365874961 3.51067865743283 0.964601072740988 0.754103646702673 1.1672351303036 1.57994332404066 1.12800699001041 2.17418796969733 0.906912888425822 1.15581187460071 2.85083277089491 0.999382957424427 2.16548212851749 1.70315860845141 2.7576632156502 2.49334387112619 0.635210499714428 0.781531715515684 1.12271241005961 2.53175233219939 3.42960156233565 2.63390861933082 0.966615097980707 1.2719710787249 0.998635660370115 1.29353608769697 2.98557564363897 2.69389070492951 1.84524593664795 0.776613332906396 1.95381256935932 0.943160932306396 1.67042482733867 1.14006692002348 1.49970746249906 2.20141315669143 0.6085415509004 1.8030751178133 1.45227231816018 1.42325012319912 1.23519373025731 1.7368062004057 1.32894631421807 2.98423377898215 1.01365834346028 1.25349787862432 2.36248253076129 3.11297352492337 2.33817908989281 1.06956242970734 3.14572625991456 1.78608526255804 3.06614260378532 0.777717112100677 1.75863706316503 2.36382177164726 ] ;
#[ 4.12897118900143 3.31208607587473 2.38807179765043 3.41147748258229 3.79245079883684 2.13747280762826 3.44956585218633 5.28646200487258 4.03521223718745 3.25010095217548 2.83783752601021 2.6258688233986 4.15775547914634 3.19704381968787 3.49010359449681 1.79747672800912 2.79288355209615 5.26616331769092 3.35371702307834 4.26724092359593 4.12133571387829 4.02973593424951 4.37931024301281 4.00283132031464 3.28692954788353 5.02754967972644 2.45016971968765 3.08356132249523 4.73987670721116 2.39922887851403 2.27569304736447 3.28007996822294 2.54369893421692 2.90570860664853 4.33460855979281 2.10734212190294 3.94274864584657 2.94933175486612 2.96797798805946 4.49957161364253 3.55441597247401 3.5759179615567 5.22306192290658 3.42170022535441 3.76491150247431 4.48288020276711 4.52788910657265 3.34478826332314 1.89566269141826 4.18464187682972 2.64100205563702 2.90739230913531 3.11192934889936 3.15676430313376 3.50731314775469 2.43757256837127 4.40912368349102 3.05764917860376 2.69090410017207 3.3205930925394 2.42452551665285 4.30593061757103 4.33316152627507 3.92695975697722 4.37897749836831 2.83479500224315 2.53467383882391 3.51459224977319 2.20466402457844 1.6221429978863 3.78082809236053 5.28887059090758 2.52449966161858 3.3807657995223 4.24820153750904 3.38856126203602 3.47046392829381 4.18450545204161 2.86556922854502 3.64298488041278 3.95812567089706 2.35633224757615 3.75879459842517 3.40005193538151 3.57065321381565 3.11571762890401 1.74034797549426 2.85477304765497 3.18978970168502 2.81230278117855 2.35127310541626 3.04778140549825 3.00467357080443 4.29845830901203 3.43788484999993 3.26446188116047 1.35633393660201 3.3205149317633 3.81553195472428 4.79030716957375 1.6451672128233 0.90153342720961 1.2308988709723 0.931449095062756 1.11946697711616 1.22740828467123 2.3788570154965 1.30174755790765 1.68596453477043 2.28141876612837 1.52928800234292 0.979164151857376 1.69693306967507 1.26921427186122 1.66929906850498 1.31169326092869 1.10317770135736 1.78737168658414 2.62485102201305 2.20692070351376 1.28883078610861 1.34746353396602 1.29042106125579 0.790791668673596 1.09861738257967 1.0111428996338 1.36805389933401 1.57532963235617 1.48102676396049 1.68286684065163 1.7282850605289 1.29953965017855 0.74271358657834 2.1624740607934 1.33572786228747 2.75966918167155 1.49656879690113 0.253282934427375 1.05012521382531 1.44355982196381 1.63413776181899 1.59305207682708 1.9836696117007 1.80853094914091 0.844314391813334 2.04410398934248 0.796089508165719 2.06962883369405 1.57301150587759 0.183616404353888 1.54400234569897 2.2298457013065 1.0490952990934 0.62170877311536 1.20914361982649 0.88145314605249 0.68356438973477 0.100513782239737 2.46252503958821 3.47953383439018 1.18537380334849 1.39083890154747 0.734637566681813 1.31822731226033 1.88590573599412 0.889515143229485 0.942743394047482 2.55250738056366 1.44048841946934 0.662057932814987 2.55200343487939 1.55182835980464 0.336100297643925 2.26751255971095 0.66582937784316 1.20032180487943 0.995391920674359 0.647631137389384 0.26008770316583 1.77958920397643 2.33496032194045 0.327511074553134 0.409963986465482 1.310835288062 0.813196786324582 1.59693394454067 0.732359096568661 1.40446379716438 0.932793779606208 0.684933583409218 0.724840214933873 2.06660009627602 1.12237016621372 1.42368984635703 1.41260314794412 1.73761700826276 1.36707058852145 2.23613030792824 1.3008181333711 0.630030371603664 ] 
#]
clusterDistances = clusterDists1

#variable allowing user to set a percentage +/- around the target number of members of each cluster
varianceFactor=0.9

NUMCLUSTERS = size(clusterDistances)[1]
NUMPOINTS = size(clusterDistances)[2]

#penalty for including a point from a different subcluster
pen = sum(sum(clusterDistances))

#assign penalties
subclusterPenalties = zeros(
    NUMCLUSTERS,
    NUMPOINTS
)
for clusterId in 1:NUMCLUSTERS
    for pointId in 1:NUMPOINTS
        if (subclusterCosts[clusterId,pointId] != clusterId)
            subclusterPenalties[clusterId,pointId] = pen^subclusterCosts[clusterId,pointId]
        end
    end
end


#variables for the selection of points in clusters
#the model will use these values to record which cluster a point is assigned to
@variable(aMIPModel, 0 <= inRoutes[1:NUMCLUSTERS,1:NUMPOINTS] <= 1, Int)

#add constraints to prevent any point being in more than one cluster 
for i = 1:NUMPOINTS
    @constraint(aMIPModel, sum(sum(inRoutes[1:NUMCLUSTERS,i])) == 1)
end

#add constraints to control the size of the clusters within a range of variance specified earlier
midPoint = NUMPOINTS/NUMCLUSTERS
lowPoint = midPoint*(1-varianceFactor)
highPoint = midPoint*(1+varianceFactor)
for i = 1:NUMCLUSTERS
    @constraint(aMIPModel, lowPoint <= sum(sum(inRoutes[i,1:NUMPOINTS])) <= highPoint)
end

#sum the distances and penalties in clusters
exprDists = 0
for clusterId = 1:NUMCLUSTERS
    for pointId = 1:NUMPOINTS
        exprDists = exprDists + ( (clusterDistances[clusterId,pointId]+subclusterPenalties[clusterId,pointId]) * inRoutes[clusterId,pointId])
    end
end


#create the final objective function
expr = exprDists 
@objective(aMIPModel, Min, expr)
println("model built")

model built


In [4]:
@time begin
    status = solve(aMIPModel)
end
println("Objective value: ", getobjectivevalue(aMIPModel))


  5.048162 seconds (8.48 M allocations: 418.198 MiB, 5.60% gc time)
Objective value: 1.805961055074695e10


In [5]:
using DelimitedFiles
writedlm("/home/james/test.txt", getvalue(inRoutes), ",")